In [18]:
import dataclasses
import typing
import enum
from typing import Any

class Status(enum.Enum):
    Ok = enum.auto()
    Err = enum.auto()

class ReturnStatus:
    status = None
    def is_okay(self) -> bool:
        return self.status is Status.Ok
    def handle(self, 
            err_funcs: typing.Dict[enum.Enum, typing.Callable[[Any],Any]],
            okay_func: typing.Callable[[Any],Any] = lambda x: x,
        ) -> typing.Any:
        if self.is_okay():
            return okayfunc(self.data)
        else:
            return err_funcs[self.error_type](*self.args, **self.kwargs)

@dataclasses.dataclass
class Ok(ReturnStatus):
    data: typing.Any
    status: Status = Status.Ok

class Err(ReturnStatus):
    def __init__(self,
        error_type: typing.Any,
        *args,
        status: Status = Status.Err,
        **kwargs,
    ):
        self.error_type = error_type
        self.status = status
        self.args = args
        self.kwargs = kwargs

    
class ErrorType(enum.Enum):
    MISSING = enum.auto()
    NOT_EVEN = enum.auto()
    
@dataclasses.dataclass
class MyType:
    a: int
    def access_a(self) -> ReturnStatus:
        if self.a is None:
            return Err(ErrorType.MISSING)
        elif self.a % 2 == 0:
            return Ok(self.a)
        else:
            return Err(ErrorType.NOT_EVEN)

def raise_(exception: BaseException):
    raise exception

fme = 54
a = MyType(1).access_a().handle({
    ErrorType.MISSING: lambda: raise_(ValueError()), 
    ErrorType.NOT_EVEN: lambda: None,
})
a

True

In [ ]:
import statistics

